In [1]:
import skimage.io as io
import matplotlib.pyplot as plt
from PIL import Image, ImageChops
import numpy as np
import os
import cv2

def cropAlign(img):
    w,h=img.shape #I found photo's size with that
    
    lenght=int(w/3) #To crop I splitted width value
    blue=img[:lenght]
    green=img[lenght:2*lenght]
    red=img[2*lenght:3*lenght]
    #I splitted by three with using array 

    cVar1=0.05
    cVar2=0.075
    #I evaluated all possible clipping cases using a for loop to find the most appropriate clipping value
    #and found these optimal values.

    blue=blue[int(cVar1*len(blue)):-int(cVar2*len(blue)),int(cVar1*len(blue[0])):-int(cVar2*len(blue[0]))]
    red=red[int(cVar1*len(red)):-int(cVar2*len(red)),int(cVar1*len(red[0])):-int(cVar2*len(red[0]))]
    green=green[int(cVar1*len(green)):-int(cVar2*len(green)),int(cVar1*len(green[0])):-int(cVar2*len(green[0]))]

    stack = (np.dstack((red,green,blue))).astype(np.uint8) #I combined them all     

    gbAlign= align(blue,green,15) #Make align operation Green to Blue
    rbAlign = align(blue,red,15) #Make align operation Red to Blue
    g=np.roll(green,gbAlign,axis=(0,1))
    r=np.roll(red,rbAlign,axis=(0,1))

    aligned = (np.dstack((r,g,blue))).astype(np.uint8) #I combined them all after new situation

    aligned=aligned[int(aligned.shape[0]*0.05):-int(aligned.shape[0]*0.015),int(aligned.shape[1]*0.04):-int(aligned.shape[1]*0.01)]
    #I also did some small cropping on the last photo I combined to fix the color problems around the edges.
    aligned = Image.fromarray(aligned)
    return aligned

def ncc(var1,var2): #I wrote the ncc algorithm I showed in the report as a code
    var1=var1-var1.mean(axis=0)
    var2=var2-var2.mean(axis=0)
    return np.sum(((var1/np.linalg.norm(var1))*(var2/np.linalg.norm(var2))))

def align(var1, var2, r):
    nccMin = -1
    iVal=np.linspace(-r,r,2*r,dtype=int)
    jVal=np.linspace(-r,r,2*r,dtype=int)   
    for i in iVal: #I used the appropriate binary for loop with [-15,15] pixel size.
        for j in jVal:
            nccDiff = ncc(var1,np.roll(var2,[i,j],axis=(0,1)))
            if nccDiff>nccMin:
                nccMin=nccDiff
                result=[i,j]           
    return result

def gammaCorr(img,gamma): #I created table that maps and I set up pixel values to [0, 255]
    varGamma =1.0/gamma
    table = np.array([((i / 255.0)**varGamma)*255
        for i in np.arange(0, 256)]).astype("uint8")
    return cv2.LUT(img, table) 


def histogramEq(img):
    yuv = cv2.cvtColor(img, cv2.COLOR_BGR2YUV) #I changed the photo to yuv type
    yuv[:,:,0] = cv2.equalizeHist(yuv[:,:,0])
    colored = cv2.cvtColor(yuv, cv2.COLOR_YUV2BGR)
    #After histogram equalization
    #I converted the photo to its original state
    return colored


#I stored the images created after Align under the Result-Part2 folder.
if not os.path.exists('../html/Result-Part2'): 
        os.makedirs('../html/Result-Part2')
        
for filename in os.listdir("../html/data"):
    img=io.imread("../html/data/"+filename)
    coloured=cropAlign(img)
    coloured.save('../html/Result-Part2/'+filename)

#I stored the result images under the Result-Part3 folder.
if not os.path.exists('../html/Result-Part3'):
        os.makedirs('../html/Result-Part3')
        
for filename in os.listdir("../html/Result-Part2"):    
    img2=io.imread("../html/Result-Part2/"+filename)
    coloured2=gammaCorr(img2,0.6)
    coloured2=histogramEq(coloured2)
    data = Image.fromarray(coloured2)
    data.save('../html/Result-Part3/'+filename)
